Analyzing the NYC Subway Dataset
=================

Secction 0. References
---------------------

* [Welch's t test](http://en.wikipedia.org/wiki/Welch%27s_t_test)
* [Mann-Whitney U test](http://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test)
* [Coefficient of determination](http://en.wikipedia.org/wiki/Coefficient_of_determination)

Section 1. Statistical Test
---------------------------

In [1]:
import numpy as np
import pandas
import scipy.stats

df = pandas.read_csv("data/turnstile_data_master_with_weather.csv")

rain_entries = df[df.rain==1].ENTRIESn_hourly
no_rain_entries = df[df.rain==0].ENTRIESn_hourly

rain_mean = np.mean(rain_entries)
no_rain_mean = np.mean(no_rain_entries)
U, p = scipy.stats.mannwhitneyu(rain_entries, no_rain_entries)

print "Mean number of riders on rainy day: ", rain_mean
print "Mean number of riders on non-rainy day: ", no_rain_mean
print "p-value for Mann-Whitney U test: ", p

Mean number of riders on rainy day:  1105.44637675
Mean number of riders on non-rainy day:  1090.27878015
p-value for Mann-Whitney U test:  0.0249999127935


### 1.1

I used Mann-Whitney U Test to analyze the NYC subway data.

I used a two-tail P value.

The null hypothesis is the subway riders are not affected by whether it rains or not, i.e. the number of riders on rainy days and number of riders on non-rainy days follow the same distribution.

The P-critical value I use is 5%.

### 1.2

The Mann-Whitney U Test is applicable to the dataset is a nonparametric test that does not make any assumption on the underlying distribution. By ploting the histogram of two ridership samples, we see they are clearly not normal distribution, and therefore a t test is not suitable for this dataset.

### 1.3

The results we get from this statistical test is that the ridership **is** affected by whether it rains or not, in other words, the number of riders on rainy days and number of riders on non-rainy days **do not** follow the same distribution. Furthermore, the number of riders on a rainy day is slightly larger than that on a non-rainy day.

The p-value of this test is 0.025. The mean number of riders on rainy days is 1105, and that on non-rainy days is 1090.

### 1.4

The significance level of this test is 5% (the P-critical value I set in 1.1). The interpretation is that the probability of our conclusion is wrong because of bad luck is less than 5%. More specifically, if the ridership is not affected by raining or not, the probability of seeing data as extreme as the one we currently have is less than 5%.

Section 2. Linear Regression
---------------------------

### 2.1

I used both (a) OLS with statsmodels and (b) gradient descent with skikit learn.

In [3]:
import statsmodels.api as sm


def linear_regression_ols(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features
    using ordinary least squares.
    """
    
    features = sm.add_constant(features)
    model = sm.OLS(values, features)
    results = model.fit()
    intercept = results.params[0]
    params = results.params[1:]
    
    return intercept, params

In [4]:
import numpy as np
from sklearn.linear_model import SGDRegressor


def normalize_features(features):
    ''' 
    Returns the means and standard deviations of the given features, along with a normalized feature
    matrix.
    ''' 
    means = np.mean(features, axis=0)
    std_devs = np.std(features, axis=0)
    normalized_features = (features - means) / std_devs
    return means, std_devs, normalized_features


def recover_params(means, std_devs, norm_intercept, norm_params):
    ''' 
    Recovers the weights for a linear model given parameters that were fitted using
    normalized features. Takes the means and standard deviations of the original
    features, along with the intercept and parameters computed using the normalized
    features, and returns the intercept and parameters that correspond to the original
    features.
    ''' 
    intercept = norm_intercept - np.sum(means * norm_params / std_devs)
    params = norm_params / std_devs
    return intercept, params


def linear_regression_sgd(features, values, n_iter=20):
    """
    Perform linear regression given a data set with an arbitrary number of features
    using stochastic gradient descent algorithm.
    """
    
    means, std_devs, normalized_features = normalize_features(features)
    clf = SGDRegressor(n_iter=n_iter)
    fit_result = clf.fit(normalized_features, values)
    norm_intercept = fit_result.intercept_[0]
    norm_params = fit_result.coef_
    intercept, params = recover_params(means, std_devs, norm_intercept, norm_params)
    
    return intercept, params

### 2.2

I use `hour`, `weekday` and `meantempi` input variables together with `UNIT` and `conds` as features for linear regression.

In [32]:
import pandas
import numpy as np

def compute_r_squared(data, predictions):    
    SST = ((data - np.mean(data)) ** 2).sum()
    SSReg = ((predictions-data) ** 2).sum()
    r_squared = 1 - SSReg / SST
    
    return r_squared


df = pandas.read_csv("data/turnstile_weather_v2.csv")

# Select features
features = df[['hour', 'weekday', 'meantempi']]
dummy_units = pandas.get_dummies(df['UNIT'], prefix='unit')
features = features.join(dummy_units)
dummy_conds = pandas.get_dummies(df['conds'], prefix='conds')
features = features.join(dummy_conds)

# Values
values = df['ENTRIESn_hourly']

# Perform linear regression
intercept, params = linear_regression_ols(features.values, values.values)
predictions = intercept + np.dot(features.values, params)

predictions = predict(df)
r2 = compute_r_squared(values, predictions)

print "Coefficients of non-dummy features: ", params[:3]
print "Coefficient of determination (R^2):", r2

Coefficients of non-dummy features:  [ 123.47256977  948.39682638  -21.12298936]
Coefficient of determination (R^2): 0.476284384231


### 2.3

I decide to use `weekday` and `hour` because I think the number of riders should be strongly affect by time, e.g. there should be more riders on weekday rush hours (say Monday 9am) than other times (say Saturday 4pm).

Similarly, I use `meantempi` and `conds` because I think the weather should also affect number of riders, e.g. there might be more riders on a cold rainy day than a warm sunny day.

Finally, I find that `UNIT` has biggest impact to the ridership as adding it significantly boost the R<sup>2</sup> value.

### 2.4

The coefficients for non-dummy features are `[ 123.47256977  948.39682638  -21.12298936]`, corresponding to `hour`, `weekday` and `meantempi`.

### 2.5

The R<sup>2</sup> value of the model is `0.47628`.

### 2.6

Having a large R<sup>2</sup> value (close to `1.0`) indicates there is strong linear relationship between the features we chose and the ridership, and vice ver sa. I think a R<sup>2</sup> value of `0.47628` is relatively small, because we are using relatively small number of features to explain the complicated ridership trend. I think the R<sup>2</sup> value we get is reasonable, and it suggests linear model might not be the most appropriate one and a more complicated model is likely to yield better prediction results.

Section 3. Visualization
-----------------------

Section 4. Conclusion
--------------------

Section 5. Reflection
--------------------